In [ ]:
!nvidia-smi

Sun Jun 13 07:42:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q git+https://github.com/ChristophReich1996/Involution
!pip install -q gradio

     |████████████████████████████████| 1.5MB 15.8MB/s 
     |████████████████████████████████| 1.9MB 49.7MB/s 
     |████████████████████████████████| 215kB 50.0MB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
     |████████████████████████████████| 962kB 48.6MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 


In [ ]:
# Import necessary libraries

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from torchvision.datasets import CIFAR10
from torchvision import datasets, transforms
from torch.optim import *

import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import cv2
import glob
import copy

from sklearn.model_selection import train_test_split
#import wandb

from torchsummary import summary

from skimage.feature import hog
from tqdm import tqdm as tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

seed = 42

from numba import jit, cuda

import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset

from PIL import Image
import glob

import albumentations as A
from albumentations.pytorch import ToTensor


import torch
from involution import Involution2d

#involution = Involution2d(in_channels=32, out_channels=64)
#output = involution(torch.rand(1, 32, 128, 128))

In [ ]:
# Mount google drive to colab

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
class Block_en(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding = 1)
        self.inonv1 = Involution2d(in_channels=in_ch, out_channels=out_ch, kernel_size = (3,3), padding = (1,1))
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding = 1)
        self.inonv2 = Involution2d(in_channels=out_ch, out_channels=out_ch, kernel_size = (3,3), padding = (1,1))
    
    def forward(self, x):
        #return self.relu(self.conv2(self.relu(self.conv1(x))))
        #print(self.inonv1(x).shape)
        return self.relu(self.conv2(self.relu(self.inonv1(x))))
        #return self.relu(self.inonv2(self.relu(self.inonv1(x))))

In [ ]:
class Block_de(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding = 1)
        #self.inonv1 = Involution2d(in_channels=in_ch, out_channels=out_ch, kernel_size = (3,3), padding = (1,1))
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding = 1)
        #self.inonv2 = Involution2d(in_channels=out_ch, out_channels=out_ch, kernel_size = (3,3), padding = (1,1))
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))
        #print(self.inonv1(x).shape)
        #return self.relu(self.conv2(self.relu(self.inonv1(x))))

In [ ]:
class Encoder(nn.Module):
    def __init__(self, chs=(3,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block_en(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs

In [ ]:
class Decoder(nn.Module):
    def __init__(self, chs=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.chs         = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block_de(chs[i], chs[i+1]) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs   = torchvision.transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs

In [ ]:
class InvolutionUNet(nn.Module):
    def __init__(self, enc_chs=(3,64,128,256,512), dec_chs=(512, 256, 128, 64), num_class=1, retain_dim=False, out_sz=(572,572)):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)
        self.retain_dim  = retain_dim
        self.out_sz = out_sz

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, self.out_sz)
        return out


## Inference on the Best Model 

In [ ]:
test_transform = A.Compose(
    [
        #A.CenterCrop(height=224, width=224),
        A.Resize(height=720, width=720),
        A.Normalize(mean=(0.64, 0.6, 0.58),std=(0.14,0.15, 0.152)),
        ToTensor(),
    ])

In [ ]:
best_model = InvolutionUNet(enc_chs=(3,64,128,256), dec_chs=(256, 128, 64), num_class=3, retain_dim=False, out_sz=(128,128))

best_ckp = torch.load('/content/drive/MyDrive/CV_Project/CheckPoints/r3/unet-14.pt')

best_model.load_state_dict(best_ckp['model'])

<All keys matched successfully>

In [ ]:
!ls -lrt

total 8
drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
drwx------ 6 root root 4096 Jun 13 07:43 drive


In [ ]:
'''
image_hazy_path = '/content/kate-joie-a8zRxmKSnJ4-unsplash.jpg'
hazy = cv2.imread(image_hazy_path)
hazy = cv2.cvtColor(hazy, cv2.COLOR_BGR2RGB)
'''

"\nimage_hazy_path = '/content/kate-joie-a8zRxmKSnJ4-unsplash.jpg'\nhazy = cv2.imread(image_hazy_path)\nhazy = cv2.cvtColor(hazy, cv2.COLOR_BGR2RGB)\n"

In [ ]:
'''
plt.imshow(hazy)
'''

'\nplt.imshow(hazy)\n'

In [ ]:
def dehaze_image(hazy_image, model, transform):
  mask_dummy = np.zeros_like(hazy_image)
  transformed = transform(image=hazy_image, mask=mask_dummy)
  #hazy = cv2.imread(image_hazy_path)
  #hazy = cv2.cvtColor(hazy, cv2.COLOR_BGR2RGB)
  hazy_image_transformed = transformed['image']
  model.eval()
  with torch.no_grad():
    hazy_image_tensor = torch.tensor(hazy_image_transformed).to(device).unsqueeze(0)
    model = model.to(device)
    #print(hazy_image.shape)
    pred_mask_tensor = model(hazy_image_tensor)
    #print(pred_mask_tensor.shape)
    pred_mask_tensor_final = torch.reshape(pred_mask_tensor, hazy_image_tensor.shape)
    #print(pred_mask_tensor_final.shape)
        #maxValue = np.amax(pred_masks_fod.detach().cpu().numpy())
        #minValue = np.amin(pred_masks_fod.detach().cpu().numpy())
    
    pred_mask = (np.clip(pred_mask_tensor_final.detach().cpu().squeeze(0).permute(1,2,0).numpy(), 0 , 1)*255).astype('uint8')
    #print(pred_mask.shape)
  return pred_mask

In [ ]:
'''
dehazed = dehaze_image(hazy, best_model, test_transform)
'''

'\ndehazed = dehaze_image(hazy, best_model, test_transform)\n'

In [ ]:
'''
plt.imshow(dehazed)
'''

'\nplt.imshow(dehazed)\n'

In [ ]:
import gradio as gr


Dehazer = lambda hazy : dehaze_image(hazy, best_model, test_transform)

iface = gr.Interface(fn=Dehazer, 
                     inputs= gr.inputs.Image(label = 'Hazy Image'),
                     outputs = gr.outputs.Image(label = 'Dehazed Image'),
                     live = True,
                     title = "Efface the haze - Demo",
                     #description = 'This application removes haze and generates a clear picture',
                     allow_flagging = False,
                     )
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://31565.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://31565.gradio.app')

In [ ]:
print("Notebook running")

Notebook running
